In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Function to calculate monthly seasonality
def calculate_monthly_seasonality(stock_symbol, n_years, include_all_time=True):
    current_year = datetime.now().year

    # Download historical stock data since inception and exclude the current year
    stock_data_all = yf.download(stock_symbol, period="max")
    stock_data_all = stock_data_all[stock_data_all.index.year < current_year].copy()  # Use .copy()

    # Calculate daily returns
    stock_data_all['Daily Return'] = stock_data_all['Adj Close'].pct_change()

    # Extract the year and month from the date
    stock_data_all['Year'] = stock_data_all.index.year
    stock_data_all['Month'] = stock_data_all.index.month

    # Calculate monthly returns for all-time data
    if include_all_time:
        monthly_returns_all = stock_data_all.groupby(['Year', 'Month'])['Daily Return'].mean()
    else:
        monthly_returns_all = None  # No all-time data when not checked

    # Download historical stock data for the last n years and exclude the current year
    stock_data_n_years = yf.download(stock_symbol, period=f"{n_years}y")
    stock_data_n_years = stock_data_n_years[stock_data_n_years.index.year < current_year].copy()  # Use .copy()

    # Calculate daily returns
    stock_data_n_years['Daily Return'] = stock_data_n_years['Adj Close'].pct_change()

    # Extract the year and month from the date
    stock_data_n_years['Year'] = stock_data_n_years.index.year
    stock_data_n_years['Month'] = stock_data_n_years.index.month

    # Calculate monthly returns for n-year data
    monthly_returns_n_years = stock_data_n_years.groupby(['Year', 'Month'])['Daily Return'].mean()

    return monthly_returns_all, monthly_returns_n_years

# Function to plot the seasonality results
def plot_seasonality(stock_symbol, monthly_returns_all, monthly_returns_n_years, n_years, include_all_time):
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Calculate and plot all-time data if available and the checkbox is checked
    if include_all_time and monthly_returns_all is not None:
        average_monthly_returns_all = monthly_returns_all.groupby('Month').mean()
        gain_frequency_all = monthly_returns_all.groupby('Month').apply(lambda x: (x > 0).mean())
        ax1.bar(average_monthly_returns_all.index, average_monthly_returns_all * 100, alpha=0.7, color='b', label='All-Time Average Monthly Gain')

    # Calculate and plot n-year data
    average_monthly_returns_n_years = monthly_returns_n_years.groupby('Month').mean()
    gain_frequency_n_years = monthly_returns_n_years.groupby('Month').apply(lambda x: (x > 0).mean())
    ax1.bar(average_monthly_returns_n_years.index, average_monthly_returns_n_years * 100, alpha=0.7, color='g', label=f'{n_years}-Year Average Monthly Gain')
    
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Average Gain (%)')
    ax1.set_xticks(average_monthly_returns_n_years.index)
    ax1.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

    # Create second y-axis for gain frequency
    ax2 = ax1.twinx()

    if include_all_time and monthly_returns_all is not None:
        ax2.plot(gain_frequency_all.index, gain_frequency_all, color='r', marker='o', label='All-Time Gain Frequency')
    
    ax2.plot(gain_frequency_n_years.index, gain_frequency_n_years, color='m', marker='o', label=f'{n_years}-Year Gain Frequency')
    ax2.set_ylabel('Gain Frequency')
    ax2.set_ylim(0, 1)

    plt.title(f'Monthly Seasonality Analysis for {stock_symbol}')
    plt.grid(True)
    fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
    plt.show()

# Function to handle button click and display the chart
def on_button_click(b):
    stock_symbol = ticker_input.value.upper()  # Get the user input ticker symbol
    try:
        n_years = int(n_years_input.value)  # Get the user input for the number of years
    except ValueError:
        with output:
            clear_output(wait=True)
            print("Please enter a valid number of years.")
        return

    include_all_time = all_time_checkbox.value  # Check if "Show All Time Data" is selected

    if stock_symbol and n_years > 0:
        with output:
            clear_output(wait=True)
            print(f"Generating seasonality analysis for {stock_symbol} over the last {n_years} years...")
            try:
                monthly_returns_all, monthly_returns_n_years = calculate_monthly_seasonality(stock_symbol, n_years, include_all_time)
                plot_seasonality(stock_symbol, monthly_returns_all, monthly_returns_n_years, n_years, include_all_time)
            except Exception as e:
                print(f"An error occurred: {e}")
    else:
        with output:
            clear_output(wait=True)
            print("Please enter a valid stock ticker symbol and number of years.")

# Create widgets for ticker input, n years input, checkbox, and generate button
ticker_input = widgets.Text(
    value='',
    placeholder='Enter stock ticker symbol',
    description='Ticker:',
    disabled=False
)

n_years_input = widgets.Text(
    value='5',
    placeholder='Enter number of years',
    description='Years:',
    disabled=False
)

all_time_checkbox = widgets.Checkbox(
    value=True,
    description='Show All Time Data?',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='success',
    tooltip='Click to generate the seasonality analysis',
    icon='line-chart'
)

generate_button.on_click(on_button_click)

# Output area for displaying the chart
output = widgets.Output()

# Display widgets and output area
display(ticker_input, n_years_input, all_time_checkbox, generate_button, output)


C:\Users\brand\anaconda3_2\Lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Text(value='', description='Ticker:', placeholder='Enter stock ticker symbol')

Text(value='5', description='Years:', placeholder='Enter number of years')

Checkbox(value=True, description='Show All Time Data?')

Button(button_style='success', description='Generate', icon='line-chart', style=ButtonStyle(), tooltip='Click …

Output()